In [1]:
import numpy as np
import torch
import h5py
import torch.nn as nn
import torch_geometric
import torch.nn.functional as F
import torch_geometric.nn as PyG
from torch_geometric.transforms import Distance
from torch_geometric.data import DataLoader
from torch_geometric.data import Data as PyGData
from torch_geometric.data import Data
import sys, os
import subprocess
import csv, yaml
import math
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import torch.optim as optim
import argparse
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/delphes'
graph_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/graph/'
output_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex2/220323_084833/h5/'

# path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/delphes'
# graph_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/graph/'
# output_path = '/users/yewzzang/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/4top_dump_ex1/220324_055935/h5/'



In [4]:
res = [] 
for root, dirs, files in os.walk(graph_path):
    rootpath = os.path.join(os.path.abspath(graph_path), root)
    for file in files:
        filepath = os.path.join(rootpath, file)
        
        filetype = filepath.split('.')[-1]
        if filetype != 'h5': continue
  
        res.append(filepath)

In [5]:
file = h5py.File(res[0],'r')

In [27]:

datalist = []
for i in range(len(file['events']['m'])):
# for i in range(1000):
    feature = torch.Tensor((np.ones(len(file['events']['m'][i]))*np.array(file['events']['id'][i])).reshape(-1,1))

    if len(feature[(torch.abs(feature) > 10) & (torch.abs(feature)<20)]) >0:
        continue
    else:
        status = torch.Tensor(np.array(file['events']['status'][i]).reshape(-1,1))

        weight = torch.Tensor((np.ones(len(file['events']['m'][i]))*np.array(file['events']['weight'][i])).reshape(-1,1))

        px = torch.Tensor(np.array(file['events']['px'][i]).reshape(-1,1))
        py = torch.Tensor(np.array(file['events']['py'][i]).reshape(-1,1))
        pz = torch.Tensor(np.array(file['events']['pz'][i]).reshape(-1,1))

        edge = torch.Tensor(np.concatenate((np.array(file['graphs']['edge2'])[i].reshape(-1,1).T,np.array(file['graphs']['edge1'])[i].reshape(-1,1).T),axis=0)).type(dtype = torch.long)
        data = PyGData(x = feature,edge_index = edge, weight = weight)
        data.st = status
        data.px = px
        data.py = py
        data.pz = pz
 


        datalist.append(data)
    file_name = 'hadr' + res[i].split('/')[-1][:-3] + '.pt'
    torch.save(datalist, file_name)

In [39]:
pa_23_particles = np.zeros([1,3])
ne_23_particles = np.zeros([1,3])

pa_22_particles = np.zeros([1,2])
ne_22_particles = np.zeros([1,2])
for i in range(len(datalist)):
# for i in range(50):
    if len(datalist[i].x) == 23:
        if datalist[i].weight[0] < 0:
            ne_23_particles = np.concatenate([ne_23_particles,(np.concatenate([datalist[i].x[0],datalist[i].x[1],datalist[i].x[22]]).reshape(1,3))],axis=0)
    if len(datalist[i].x) == 23:
        if datalist[i].weight[0] > 0:
            pa_23_particles = np.concatenate([pa_23_particles,(np.concatenate([datalist[i].x[0],datalist[i].x[1],datalist[i].x[22]]).reshape(1,3))],axis=0)
    
    if len(datalist[i].x) == 22:
        if datalist[i].weight[0] < 0:
            ne_22_particles = np.concatenate([ne_22_particles,(np.concatenate([datalist[i].x[0],datalist[i].x[1]]).reshape(1,2))],axis=0)
    if len(datalist[i].x) == 22:
        if datalist[i].weight[0] > 0:
            pa_22_particles = np.concatenate([pa_22_particles,(np.concatenate([datalist[i].x[0],datalist[i].x[1]]).reshape(1,2))],axis=0)

In [42]:
print(pa_22_particles.shape)
print(ne_22_particles.shape)
print(pa_23_particles.shape)
print(ne_23_particles.shape)

(5482, 2)
(1225, 2)
(414, 3)
(1490, 3)


In [ ]:
a = 0 
for i in range(len(hadr)):
# for i in range(50):
    if len(hadr[i].x) == 23:
        if hadr[i].weight[0] > 0:
#             if torch.cat([hadr[i].x[0],hadr[i].x[1],hadr[i].x[22]]) == torch.Tensor([21,21,21]):
            if hadr[i].x[0] == 21:
                if hadr[i].x[1] == -5:
                    if hadr[i].x[22] == -5:
    #                     print(torch.cat([hadr[i].x[0],hadr[i].x[1],hadr[i].x[22]]))
        #                 print(torch.cat([hadr[i].x[0],hadr[i].x[1]]))
                        plot_graph_test(hadr[i])
                        a = a + 1
print(a)

In [ ]:
ne = 0
ne_23 = 0
ne_22 = 0
for i in range(len(hadr)):
    if hadr[i].weight[0] < 0:
        ne = ne +1
    else:
        continue
    if len(hadr[i].weight) ==22:
        ne_22 = ne_22 + 1
    else:
        ne_23 = ne_23 +1

In [ ]:
pa = 0
pa_23 = 0
pa_22 = 0
for i in range(len(hadr)):
    if hadr[i].weight[0] > 0:
        pa = pa +1
    else:
        continue
    if len(hadr[i].weight) ==22:
        pa_22 = pa_22 + 1
    else:
        pa_23 = pa_23 +1

In [ ]:
pa_23_particles = np.zeros([1,3])
ne_23_particles = np.zeros([1,3])

pa_22_particles = np.zeros([1,2])
ne_22_particles = np.zeros([1,2])
for i in range(len(hadr)):
# for i in range(50):
    if len(hadr[i].x) == 23:
        if hadr[i].weight[0] < 0:
            ne_23_particles = np.concatenate([ne_23_particles,(np.concatenate([hadr[i].x[0],hadr[i].x[1],hadr[i].x[22]]).reshape(1,3))],axis=0)
    if len(hadr[i].x) == 23:
        if hadr[i].weight[0] > 0:
            pa_23_particles = np.concatenate([pa_23_particles,(np.concatenate([hadr[i].x[0],hadr[i].x[1],hadr[i].x[22]]).reshape(1,3))],axis=0)
    
    if len(hadr[i].x) == 22:
        if hadr[i].weight[0] < 0:
            ne_22_particles = np.concatenate([ne_22_particles,(np.concatenate([hadr[i].x[0],hadr[i].x[1]]).reshape(1,2))],axis=0)
    if len(hadr[i].x) == 22:
        if hadr[i].weight[0] > 0:
            pa_22_particles = np.concatenate([pa_22_particles,(np.concatenate([hadr[i].x[0],hadr[i].x[1]]).reshape(1,2))],axis=0)

In [ ]:
ne_23_particles = ne_23_particles[1:]
pa_23_particles = pa_23_particles[1:]
ne_22_particles = ne_22_particles[1:]
pa_22_particles = pa_22_particles[1:]

In [ ]:
uni_ne_23 = np.unique(ne_23_particles,axis=0)
uni_pa_23 = np.unique(pa_23_particles,axis=0)
uni_ne_22 = np.unique(ne_22_particles,axis=0)
uni_pa_22 = np.unique(pa_22_particles,axis=0)